Cristobal Gomez y Pepe López 

# Proyecto de predicción de contratación de productos para clientes de un banco

## Importe de librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import numpy as np
from scoring import mapk, apk


ModuleNotFoundError: No module named 'h2o'

## 1. Importación y Exploración Inicial de los Datos


### 1.1. Importación de los Datos

In [ ]:

# Cargar datos
dataset = pd.read_csv('dataset_para_modelar.csv')

### 1.2. Exploración Inicial


Mostrar información básica del dataset


In [ ]:
dataset.info()


Descripcion de las principales metricas de cada variable

In [ ]:
dataset.describe()


Primera visualizacion de las variables

In [ ]:
dataset.head(10)

## 2. Limpieza de Datos


In [ ]:
dataset.columns

### Unnamed: 0

In [ ]:
#Ver tipo 
print(dataset['Unnamed: 0'].dtypes)
#Recuento de clases 
print(dataset['Unnamed: 0'].value_counts())
#Recuento de nulos
print(dataset['Unnamed: 0'].isnull().sum())
#Ver distribucion
# plt.hist(dataset['Unnamed: 0']) Es un indice que no tiene distribiucion 
#Imputar nulos

### Cod_persona


In [ ]:
#dataset[cod_persona]=503082 y ordenarlo por fecha1:

#Ejemplo de una persona
dataset[dataset['cod_persona']==504546].sort_values(by='fecha1') # Dataset compuesto por la evolucion de contrataciones de productos de una persona a lo largo del tiempo

In [ ]:
# Agrupar por 'cod_persona' y contar el número de registros para cada persona
conteo_registros = dataset.groupby('cod_persona').size().reset_index(name='num_registros')

# Filtrar para encontrar las personas con solo un registro
personas_con_un_registro = conteo_registros[conteo_registros['num_registros'] == 1]

# Contar el número de personas con solo un registro
num_personas_con_un_registro = personas_con_un_registro.shape[0]

print(num_personas_con_un_registro)

Se encuentran personas con un solo registro, lo que puede ser un problema predecir un solo registro sin un historico de contrataciones. No se eliminan los registros, ya que se considera que a pesar de tener un solo registro ese estado puede contener variables muy indicativas de compras de un producto nuevo la siguiente mensualidad.

In [ ]:
# Verificar si hay valores nulos en 'cod_persona'
print(dataset['cod_persona'].isnull().sum())

### mes

In [ ]:
# Paso la columna 'mes' a datetime
dataset['mes'] = pd.to_datetime(dataset['mes'], format='%Y-%m-%d')
#Recuento de nulos:
print(dataset['mes'].isnull().sum())

In [ ]:
#Extraemos las caracteristicas de mes, dia y año:
dataset['mes_mes']=dataset['mes'].dt.month
dataset['mes_año']=dataset['mes'].dt.year

In [ ]:
#Se estudia el numero de registros por mes y año para ver el balanceo de clases:
print(dataset.groupby(['mes_mes', 'mes_año']).size())

Se observa que el dataset esta balanceado en cuanto a la cantidad de registros por mes y año.
Se observa que el historico acaba en abril de 2016.

### pais

In [ ]:
print(dataset['pais'].value_counts())


Se observa como la mayoría de los registros son de España, resultando el resto de paises en cifras no significativas. Se podría considerar la posibilidad de agrupar los paises con menos registros en un grupo llamado "Otros" para reducir la dimensionalidad de la variable y se mapea a 0-1. 

In [ ]:
# Crear un diccionario de mapeo para agrupar los países
#Si pais es España el valor de la columna pais_es será 1, sino 0:
dataset['pais_binario'] = dataset['pais'].apply(lambda x: 1 if x == 'ES' else 0)
dataset.drop(columns=['pais'], inplace=True)
dataset['pais_binario'].value_counts()

In [ ]:
dataset['pais_binario'].isnull().sum() #No hay valores nulos

### sexo

In [ ]:
dataset['sexo'].isnull().sum()

In [ ]:
dataset[dataset['cod_persona']==170635]

Se observan que hay valores nulos en la columna 'sexo' que no en todos los registros de la misma persona son nulos. Suponiendo que el sexo de una persona no puede cambiar en el tiempo, se decide rellenar los valores nulos con el valor que tiene para otros registros.

In [ ]:
# Calcular el número de registros por 'cod_persona'
num_registros_por_persona = dataset.groupby('cod_persona').size().reset_index(name='total_registros')

# Calcular el número de valores nulos en la columna 'sexo' por cada 'cod_persona'
num_nulos_por_persona = dataset.groupby('cod_persona')['sexo'].apply(lambda x: x.isnull().sum()).reset_index(name='num_nulos')

# Unir la información de los nulos y el total de registros al dataset original
info_completa = num_registros_por_persona.merge(num_nulos_por_persona, on='cod_persona')

# Filtrar para encontrar las personas donde el número de nulos es igual al número de registros
personas_con_todos_nulos = info_completa[info_completa['total_registros'] == info_completa['num_nulos']]

# Mostrar el resultado
print(personas_con_todos_nulos)

In [ ]:
dataset['sexo'].value_counts()


In [ ]:
# Crear un diccionario de mapeo para asignar números a cada género
mapping_sex = {'V': 1, 'H': 0}

# Aplicar el mapeo a la columna 'customer_sex'
dataset['sexo'] = dataset['sexo'].map(mapping_sex)

In [ ]:
dataset['sexo'].value_counts()


#### Ind_proc23 y Ind_prod22

In [ ]:
dataset[['ind_prod23', 'ind_prod22']].astype('Int64')


#### xti_extra

In [ ]:
print(dataset['xti_extra'].value_counts()) #Casi todas las personas se consideran que estan vivas por lo que no aporta informacion y decide borrarse a las muertas
print(dataset['xti_extra'].isnull().sum()) #56 valores nulos.

In [ ]:
print(dataset['tip_dom'].value_counts()) #Todas las personas tienen un domicilio particular por lo que no aporta informacion

In [ ]:
print(dataset['Unnamed: 0'].value_counts()) # No aporta información por lo que son las filas que se han añadido al cargar el dataset

In [ ]:

# Eliminar columnas innecesarias
columns_to_drop = ['Unnamed: 0', 'xti_extra', 'tip_dom']

# Unnamed la elimino porque es un índice que se ha guardado en el csv
# xti_extra la elimino porque es una variable que no aporta información ya que la mayoría de los registros son personas vivas
# tip_dom la elimino porque es una variable que no aporta información ya que no aporta informacion, todas son 1

dataset.drop(columns=columns_to_drop, inplace=True)


In [ ]:
dataset['pais'].value_counts()


Se observa como la mayoría de los registros son de España, resultando el resto de paises en cifras no significativas. Se podría considerar la posibilidad de agrupar los paises con menos registros en un grupo llamado "Otros" para reducir la dimensionalidad de la variable y se mapea a 0-1. 

In [ ]:
# Crear un diccionario de mapeo para asignar números a cada género
mapping_sex = {'V': 1, 'H': 0}

# Aplicar el mapeo a la columna 'customer_sex'
dataset['sexo'] = dataset['sexo'].map(mapping_sex)

In [ ]:
dataset['sexo'].value_counts()


In [ ]:
dataset['imp_renta'].max()

In [ ]:
dataset['imp_renta'].hist()
plt.title('Distribución de imp_renta')
plt.xlabel('Ingresos Brutos (imp_renta)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Filtrar los valores extremos
q_low = dataset['imp_renta'].quantile(0.01)
q_high = dataset['imp_renta'].quantile(0.99)
filtered_data = dataset[(dataset['imp_renta'] >= q_low) & (dataset['imp_renta'] <= q_high)]

# Crear el histograma con los datos filtrados
plt.figure(figsize=(10, 6))
plt.hist(filtered_data['imp_renta'], color='blue', edgecolor='black')
plt.title('Distribución de imp_renta (valores filtrados)')
plt.xlabel('Ingresos Brutos (imp_renta)')
plt.ylabel('Frecuencia')
plt.show()

Es una distribución completamente sesgada (right-skewed) donde los valores se concentran en la parte baja de la distribución.
Podríamos considerar la posibilidad de aplicar una transformación logarítmica para reducir el sesgo y mejorar la distribución de los datos.

In [ ]:
#Recuento de nulos por columna:
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
##Notificamos que fec_ult_cli_1t tiene casi el 100% de los valores nulos, por lo que se elimina
dataset.drop(columns=['fec_ult_cli_1t'], inplace=True)

### xti_rel

In [ ]:
dataset['xti_rel'] = pd.to_numeric(dataset['xti_rel'], errors='coerce')
dataset['xti_rel'].value_counts()


In [ ]:
dataset['xti_rel'].fillna(1, inplace=True)

In [ ]:
dataset['xti_rel'] = dataset['xti_rel'].apply(lambda x: 1 if x == 1 else 0)
dataset['xti_rel'].value_counts()

### xti_rel_1mes

In [ ]:
dataset['xti_rel_1mes'] = pd.to_numeric(dataset['xti_rel_1mes'], errors='coerce')


In [ ]:
dataset['xti_rel_1mes'].value_counts()

In [ ]:
dataset['xti_rel_1mes'] = dataset['xti_rel_1mes'].apply(lambda x: 1 if x == 1 else 0)
dataset['xti_rel_1mes'].value_counts()

Casi todos los valores son 1. 

## mean_engagement, tip_rel_1mes, 

In [ ]:
dataset['mean_engagement'].value_counts()


In [ ]:
dataset['tip_rel_1mes'].value_counts()


### xti_empleado
lo consideramos como una variable binaria porque casi todos los valores se concentran en N

In [ ]:
dataset['xti_empleado'].value_counts()


In [ ]:
dataset['xti_empleado'] = dataset['xti_empleado'].apply(lambda x: 1 if x == 'N' else 0)
dataset['xti_empleado'].value_counts()

### num_antiguedad
Comprobamos la variable num_antiguedad: 

In [ ]:
dataset['num_antiguedad'] = pd.to_numeric(dataset['num_antiguedad'], errors='coerce')


In [ ]:
plt.figure(figsize=(10, 6))  # Create a figure with desired size
plt.hist(dataset['num_antiguedad'], bins=10, color='blue', edgecolor='black')  

plt.title('Distribución de antiguedad (valores filtrados)')  
plt.xlabel('Meses de antiguedad')  
plt.ylabel('Frecuencia')  

plt.show()  

In [ ]:
pd.set_option('display.max_columns', None)
print(dataset.head(5))

Calcular los valores de la moda y la mediana


ESTO ES MIERDA! NO FUNCIONA

In [ ]:
most_frequent_xti_rel_1mes = dataset['xti_rel_1mes'].mode()[0]
most_frequent_tip_rel_1mes = dataset['tip_rel_1mes'].mode()[0]
median_engagement_score = dataset['mean_engagement'].median()

Rellenar valores nulos con los valores calculados


In [ ]:
dataset['xti_rel_1mes'].fillna(most_frequent_xti_rel_1mes, inplace=True)
dataset['tip_rel_1mes'].fillna(most_frequent_tip_rel_1mes, inplace=True)
dataset['mean_engagement'].fillna(median_engagement_score, inplace=True)

Reemplazar valores nulos en cod_provincia por 0


In [ ]:
dataset['cod_provincia'].fillna(0, inplace=True)


Convertir las provincias categóricas a números


In [ ]:
dataset['cod_provincia'] = dataset['cod_provincia'].astype('category').cat.codes

In [ ]:
# Limpiar datos
dataset['edad'].fillna(dataset['edad'].mean(), inplace=True)
#Rellenamos con la media porque se considera mas representativa 
# dataset['imp_renta'].fillna(dataset['imp_renta'].median(), inplace=True) 
#Rellenamos con mediana porque existen outliers muy elevados que sesgarian el valor si rellenamos con la media los valores faltantes 

"""
Se añade el codigo para una visualizacion de la distribución logaritmica por si quisiera visualizarse
"""
# transformed_data = np.log1p(dataset['imp_renta'])  # log1p aplica log(1 + x) para manejar valores de 0
# plt.hist(transformed_data)
# plt.title('Distribución transformada de imp_renta (log)')
# plt.xlabel('Log(imp_renta)')
# plt.ylabel('Frecuencia')
# plt.show()

In [ ]:
# Feature Engineering
dataset['nueva_caracteristica'] = dataset['edad'] * dataset['imp_renta']

# Verificar cambios
print(dataset.head())

In [ ]:
# Convertir columnas con valores mixtos a numéricos
dataset['num_antiguedad'] = pd.to_numeric(dataset['num_antiguedad'], errors='coerce')
dataset['num_antiguedad'].fillna(dataset['num_antiguedad'].mean(), inplace=True)
dataset['xti_rel_1mes'] = pd.to_numeric(dataset['xti_rel_1mes'], errors='coerce')
dataset['xti_rel_1mes'].fillna(dataset['xti_rel_1mes'].mean(), inplace=True)

# Convertir columnas categóricas en variables dummy
categorical_columns = ['pais', 'sexo', 'xti_empleado', 'xti_rel', 'indresi', 'indext', 'des_canal', 'xti_extra', 'tip_rel_1mes']
dataset = pd.get_dummies(dataset, columns=categorical_columns)



dataset.columns

In [ ]:

# Verificar los cambios
print(dataset.info())
print(dataset.head())

# Separar características y targets
X = dataset.drop(columns=[f'ind_prod{i}' for i in range(1, 26)])
y = dataset[[f'ind_prod{i}' for i in range(1, 26)]]

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Entrenar el modelo
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# Generar predicciones
predicciones = modelo.predict(X_test)

# Convertir predicciones y valores reales a listas de listas para MAP@7
y_test_list = [list(np.where(row == 1)[0] + 1) for row in y_test.values]
predicciones_list = [list(np.argsort(row)[-7:][::-1] + 1) for row in predicciones]

# Calcular MAP@7
score = mapk(y_test_list, predicciones_list, 7)
print(f'MAP@7 Score: {score}')